<a href="https://colab.research.google.com/github/tuanhdao/MEDLab/blob/main/BVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Open file and delete first 5 lines which we don't need to use and save it as a new clean file
with open('Acc_segments.txt') as f: #CHANGE THIS TO YOUR FILE'S NAME
    lines = f.readlines()
    del lines[0:5]
new_file = open("Acc_Segments_clean.txt", "w+") #CHANGE THIS TO YOUR FILE'S NAME
for line in lines:
    clean_file = new_file.write(line)
new_file.close()
condition = input("Condition? This will be to name the file only. ")

Condition? This will be to name the file only. Accept


In [ ]:
#gets the number of lines and case count
file = open("Acc_Segments_clean.txt", "r")
number_of_lines = 0
case_count = 0
for line in file:
  number_of_lines += 1
#NOTE: CHANGE THIS TO YOUR ID FORMAT (i.e., if your ID is FYRE1001, change code to if "FYRE" in line)
  if "ig" in line:
    case_count += 1
length_segment = int(number_of_lines/case_count)
print("Case count: ", case_count)
print("Number of lines: ", number_of_lines)
print("Length of segment: ", length_segment)
file.close()

Case count:  407
Number of lines:  21572
Length of segment:  53


In [ ]:
#import packages
import pandas as pd
from IPython.display import display
import re

In [ ]:
#Get IDs
f = open('Acc_Segments_clean.txt') #rename to your file
count = 0  
entry = range(2, number_of_lines, length_segment)  
list_id = []
for line in f:
  count += 1
  for n in entry:
    if count == n: 
      id = line
      id_clean = id.replace("\n","") #delete trailing white space
      list_id.append(id_clean)
id_df = pd.DataFrame (list_id,columns=['ID']) #create dataframe

In [ ]:
lookup = input("Electrode (MUST BE WRITTEN AS IT IS IN THE MACRO): ")
number_segments_all_other = []
number_segments_Cz_FCz = []
#Ok now the program will look up the line associated with the chosen electrode
for line in lines:
  if lookup in line:
    value = line.strip()
    if lookup == "Cz" or lookup == "FCz": #beware of Cz and FCz because they share the same characters in the name
      number_segments_Cz_FCz.append(value) #separate list for Cz FCz values
      N=2
      sub_Cz_FCz_list = [number_segments_Cz_FCz[n:n+N] for n in range(0, 
      len(number_segments_Cz_FCz), N)] #this parses the list into smaller lists of 2 elements
      values_Cz_FCz = []
      for i in sub_Cz_FCz_list:
        for k in i:
          split_Cz_FCz = re.split(r'\t+', k)
          values_Cz_FCz.append(split_Cz_FCz[-1:])
          flat_list_Cz_FCz = []
          for sublist in values_Cz_FCz:
            for item in sublist:
              flat_list_Cz_FCz.append(item)
      sub_Cz_FCz_list_2 = [flat_list_Cz_FCz[n:n+N] for n in range(0, 
      len(values_Cz_FCz), N)] #pair the elements again to be in twos
      number_segments_df = pd.DataFrame(sub_Cz_FCz_list_2) #create a data frame
      frame = [id_df, number_segments_df]
    else:
      number_segments_all_other.append(value)
      values_all_other = []  #for all other electrodes
      for i in number_segments_all_other:
        split_other = re.split(r'\t+', i)
        values_all_other.append(split_other[-1:])
        flat_list_other = []
        for sublist in values_all_other: 
          for item in sublist:
            flat_list_other.append(item)
      number_segments_df = pd.DataFrame(flat_list_other)

#Create a list of IDs and segments
concatenate = [id_df, number_segments_df]

#merge ID and electrode values so that we add new variables, not cases
frame= pd.concat(concatenate, axis = 1) 
df = pd.DataFrame(frame)
final_renamed_df = df.copy(deep=True)

#rename columns to show which electrode is used
if lookup == "Cz" or lookup == "FCz":
  final_renamed_df.columns = ["ID", "Cz", "FCz"]
else:
  final_renamed_df.columns = ["ID", lookup]

#display the final data frame to check for accuracy)
display(final_renamed_df)

#Save to a CSV file -- CHANGE THIS
final_renamed_df.to_csv("/content/drive/MyDrive/" + "{}{}{}.csv".format(condition, "_", lookup), index = False)